In [1]:
import torch
import torchvision.models as models
from torch.autograd import Variable

inputs = torch.randn(2, 3, 224, 224)
vin1 = Variable(inputs, requires_grad=True)
vin2 = Variable(inputs[0].unsqueeze(0), requires_grad=True)
vin3 = Variable(inputs[1].unsqueeze(0), requires_grad=True)
labels = torch.LongTensor([1,0])
lab1 = Variable(labels)
lab2 = Variable(torch.LongTensor([1]))
lab3 = Variable(torch.LongTensor([0]))

net = models.resnet152(pretrained=True).eval()
loss = torch.nn.CrossEntropyLoss(size_average=False)
out1 = net(vin1)
l1 = loss(out1, lab1)
l1.backward()

v1g = vin1.grad.clone()
v1fcg = net.fc.weight.grad.clone()
v1conv1g = net.conv1.weight.grad.clone()

for p in net.parameters():
    void = p.grad.data.zero_()

out2 = net(vin2)
l2 = loss(out2, lab2)
l2.backward()
out3 = net(vin3)
l3 = loss(out3, lab3)
l3.backward()

v2g = vin2.grad.clone()
v2fcg = net.fc.weight.grad.clone()
v2conv1g = net.conv1.weight.grad.clone()
v3g = vin3.grad.clone()

In [2]:
print(l1)
print(l2)
print(l3)

def cmp(v1, v2):
    return (torch.abs(v1 - v2) > 1e-3).long().sum().data[0]
print(cmp(out1[0], out2[0]))
print(cmp(out1[1], out3[0]))
print(cmp(v1g[1], v3g[0]))
print(cmp(v1fcg, v2fcg))
print(cmp(v1conv1g, v2conv1g))

Variable containing:
 19.4826
[torch.FloatTensor of size 1]

Variable containing:
 9.1400
[torch.FloatTensor of size 1]

Variable containing:
 10.3426
[torch.FloatTensor of size 1]

0
0
0
0
0


In [ ]:
# tests when first playing around with region descriptor net (region pooling in particular)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
import random
import time
from utils import fold_batches
from model.custom_modules import TripletLoss

In [4]:
num_classes = 1000
epochs = 1
mini_batch_size = 64
micro_batch_size = 1
alpha = 1.0
image_size = (3, 300, 400)

In [5]:
dataset = [(torch.randn(*image_size), random.randrange(num_classes)) for _ in range(200)]

In [6]:
class WeirdNet(nn.Module):
    def __init__(self, net, k):
        super(WeirdNet, self).__init__()
        self.net = net
        self.k = k
        self.conv_final = nn.Conv2d(2048, 1000, (1, 1))
    
    def forward_single(self, x):
        x = self.net.conv1(x)
        x = self.net.bn1(x)
        x = self.net.relu(x)
        x = self.net.maxpool(x)
        x = self.net.layer1(x)
        x = self.net.layer2(x)
        x = self.net.layer3(x)
        
        feat = self.net.layer4(x)
        x = self.net.avgpool(feat)
        x = self.conv_final(x)
        
        c_maxv, _ = x.max(1)
        c_maxv = c_maxv.view(-1)
        k = min(c_maxv.size(0), self.k)
        _, flat_idx = c_maxv.topk(k)

        def feature_idx(flat_idx):
            cls_idx = flat_idx // x.size(3), flat_idx % x.size(3)
            return (cls_idx[0], cls_idx[0] + 7,
                    cls_idx[1], cls_idx[1] + 7)
        top_idx = [feature_idx(int(i)) for i in flat_idx.data]
        # needed for output
        tmp = c_maxv.data.clone().resize_(x.size(0), 2048)
        acc = Variable(tmp.fill_(0))
        tmp = c_maxv.data.clone().resize_(x.size(0), x.size(1), self.k)
        cls_out = Variable(tmp.fill_(0))

        i = 0
        for x1, x2, y1, y2 in top_idx:
            cls_out[:, :, i] = x[:, :, x1, y1]
            i += 1
            region = feat[:, :, x1, y1]
            acc = acc + region
        return acc, cls_out

#         k = min(self.k, x.size(2) * x.size(3))
#         tmp = Variable(feat.data.clone().resize_(feat.size(0), feat.size(1)).fill_(2))
#         cls_out = Variable(x.data.clone().resize_(x.size(0), x.size(1), k).fill_(0))
#         for i in range(k):
#             tmp = tmp + feat[:, :, random.randrange(feat.size(2)), random.randrange(feat.size(3))]
#             cls_out[:, :, i] = x[:, :, random.randrange(x.size(2)), random.randrange(x.size(3))]
#         return tmp, cls_out
    
    def forward(self, x1, x2, x3):
        return self.forward_single(x1), self.forward_single(x2), self.forward_single(x3)

In [7]:
net = models.resnet152(pretrained=True)
for layer in [net.conv1, net.bn1, net.layer1, net.layer2, net.layer3]:
    for p in layer.parameters():
        p.requires_grad = False
weird_net = WeirdNet(net, 4).cuda().train()
criterion1 = TripletLoss(0.1)
criterion2 = nn.CrossEntropyLoss()
optimizer = optim.SGD((p for p in weird_net.parameters() if p.requires_grad), lr=1e-3)

In [8]:
def micro_batch(last, i, is_final, batch):
    t1_in = torch.Tensor(len(batch), *image_size).cuda()
    t2_in = torch.Tensor(len(batch), *image_size).cuda()
    t3_in = torch.Tensor(len(batch), *image_size).cuda()
    lab1_in = torch.Tensor(len(batch)).cuda()
    lab2_in = torch.LongTensor(len(batch)).cuda()
    for j, (im, lab) in enumerate(batch):
        t1_in[j] = im
        im2 = random.choice(dataset)[0]
        t2_in[j] = im2
        im3 = random.choice(dataset)[0]
        t3_in[j] = im3
        lab1_in[j] = random.choice([1, -1])
        lab2_in[j] = lab
    t1_in = Variable(t1_in)
    t2_in = Variable(t2_in)
    t3_in = Variable(t3_in)
    lab1_in = Variable(lab1_in)
    lab2_in = Variable(lab2_in)
    out = weird_net(t1_in, t2_in, t3_in)
    loss = criterion1(out[0][0], out[1][0], out[2][0])
    cls_out = out[0][1]
    loss2 = criterion2(cls_out[:, :, 0], lab2_in)
    k = cls_out.size(2)
    for i in range(1, k):
        loss2 += criterion2(cls_out[:, :, i], lab2_in)
    loss2 /= k
    loss += alpha * loss2
    loss.backward()
    print('Epoch: {0}, Batch: {1}, Micro idx: {2}'.format(ep, last, j))
    time.sleep(4)
    return last

def mini_batch(last, i, is_final, batch):
    optimizer.zero_grad()
    fold_batches(micro_batch, last, batch, micro_batch_size, cut_end=True)
    optimizer.step()
    return last + 1

In [9]:
def create_epoch(epoch, dataset, test_set):
    return dataset, {}, {}

def create_batch(batch, n):
    t1_in = torch.Tensor(len(batch), *image_size).cuda()
    t2_in = torch.Tensor(len(batch), *image_size).cuda()
    t3_in = torch.Tensor(len(batch), *image_size).cuda()
    lab1_in = torch.Tensor(len(batch)).cuda()
    lab2_in = torch.LongTensor(len(batch)).cuda()
    for j, (im, lab) in enumerate(batch):
        t1_in[j] = im
        im2 = random.choice(dataset)[0]
        t2_in[j] = im2
        im3 = random.choice(dataset)[0]
        t3_in[j] = im3
        lab1_in[j] = random.choice([1, -1])
        lab2_in[j] = lab
    return [t1_in, t2_in, t3_in], [lab1_in, lab2_in]

def create_loss(out, labels_in):
    # out is a tuple of 3 tuples, each for the descriptor
    # and a tensor with all classification results for the highest
    # classification values. the first loss is a simple loss on the
    # descriptors. the second loss is a classification loss for
    # each sub-region of the input. we simply sum-aggregate here
    loss = criterion1(out[0][0], out[1][0], out[2][0])
    cls_out = out[0][1]
    loss2 = criterion2(cls_out[:, :, 0], labels_in[1])
    k = cls_out.size(2)
    for i in range(1, k):
        loss2 += criterion2(cls_out[:, :, i], labels_in[1])
    loss2 /= k
    return loss, loss2

def micro_batch_gen(last, i, is_final, batch):
    prev_loss, mini_batch_size = last
    n = len(batch)
    tensors_in, labels_in = create_batch(batch, n, **batch_args)
    tensors_out = weird_net(*(Variable(t) for t in tensors_in))
    loss, loss2 = create_loss(tensors_out, [Variable(l) for l in labels_in])
    loss_micro = loss * n / mini_batch_size
    val = loss.data[0]
    if loss2 is not None:
        loss2_micro = loss2 * n / mini_batch_size
        val += alpha * (loss2.data[0])
        loss_micro += alpha * loss2_micro
    loss_micro.backward()
    print('Epoch: {0}, Micro batch idx: {1}'.format(epoch, i))
    time.sleep(4)
    return prev_loss + val, mini_batch_size

def mini_batch_gen(last, i, is_final, batch):
    batch_count, score, running_loss = last
    optimizer.zero_grad()
    loss, _ = fold_batches(micro_batch_gen, (0.0, len(batch)), batch, micro_batch_size)
    optimizer.step()
    return batch_count + 1, score, running_loss

for epoch in range(epochs):
    dataset, batch_args, stats_args = create_epoch(epoch, dataset, [])

    init = 0, 0, 0.0  # batch count, score, running loss
    _, best_score, _ = fold_batches(mini_batch_gen, init, dataset, mini_batch_size, cut_end=True)

Epoch: 0, Micro batch idx: 0
Epoch: 0, Micro batch idx: 1
Epoch: 0, Micro batch idx: 2
Epoch: 0, Micro batch idx: 3
Epoch: 0, Micro batch idx: 4
Epoch: 0, Micro batch idx: 5
Epoch: 0, Micro batch idx: 6
Epoch: 0, Micro batch idx: 7
Epoch: 0, Micro batch idx: 8
Epoch: 0, Micro batch idx: 9
Epoch: 0, Micro batch idx: 10
Epoch: 0, Micro batch idx: 11
Epoch: 0, Micro batch idx: 12
Epoch: 0, Micro batch idx: 13
Epoch: 0, Micro batch idx: 14
Epoch: 0, Micro batch idx: 15
Epoch: 0, Micro batch idx: 16
Epoch: 0, Micro batch idx: 17
Epoch: 0, Micro batch idx: 18
Epoch: 0, Micro batch idx: 19
Epoch: 0, Micro batch idx: 20
Epoch: 0, Micro batch idx: 21
Epoch: 0, Micro batch idx: 22
Epoch: 0, Micro batch idx: 23
Epoch: 0, Micro batch idx: 24
Epoch: 0, Micro batch idx: 25
Epoch: 0, Micro batch idx: 26
Epoch: 0, Micro batch idx: 27
Epoch: 0, Micro batch idx: 28
Epoch: 0, Micro batch idx: 29
Epoch: 0, Micro batch idx: 30
Epoch: 0, Micro batch idx: 31
Epoch: 0, Micro batch idx: 32
Epoch: 0, Micro batc

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

In [11]:
for ep in range(epochs):
    random.shuffle(dataset)
    fold_batches(mini_batch, 0, dataset, mini_batch_size, cut_end=True)

RuntimeError: cuda runtime error (2) : out of memory at /b/wheel/pytorch-src/torch/lib/THC/generic/THCTensorMath.cu:35